# Lab 02

## Google Drive Access Setup

In [2]:
# Code to download file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

distributed 1.21.8 requires msgpack, which is not installed.
You are using pip version 10.0.1, however version 19.0.3 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


ModuleNotFoundError: No module named 'google.colab'

## Word2Vec

In [3]:
import nltk
nltk.download('punkt')

import pprint
import re
from lxml import etree
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize

from gensim.models import Word2Vec

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\daiwe\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


ModuleNotFoundError: No module named 'gensim'

### Downloading TED Scripts from Google Drive 
Click on left side "Files" tab and see the file is downloaded successfully.

In [0]:
id = '1B47OiEiG2Lo1jUY6hy_zMmHBxfKQuJ8-'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('ted_en-20160408.xml')  

### Data Preprocessing

In [0]:
targetXML=open('ted_en-20160408.xml', 'r', encoding='UTF8')

# Getting contents of <content> tag from the xml file
target_text = etree.parse(targetXML)
parse_text = '\n'.join(target_text.xpath('//content/text()'))

# Removing "Sound-effect labels" using regular expression (i.e. (Audio), (Laughter))
content_text = re.sub(r'\([^)]*\)', '', parse_text)

# Tokenising the sentence to process it by using NLTK library
sent_text=sent_tokenize(content_text)

# Removing punctuations and changing all characters to lower case
normalized_text = []
for string in sent_text:
     tokens = re.sub(r"[^a-z0-9]+", " ", string.lower())
     normalized_text.append(tokens)

# Tokenising each sentence to process individual word
sentences=[]
sentences=[word_tokenize(sentence) for sentence in normalized_text]

# Prints only 10 (tokenised) sentences
print(sentences[:10])

[['here', 'are', 'two', 'reasons', 'companies', 'fail', 'they', 'only', 'do', 'more', 'of', 'the', 'same', 'or', 'they', 'only', 'do', 'what', 's', 'new'], ['to', 'me', 'the', 'real', 'real', 'solution', 'to', 'quality', 'growth', 'is', 'figuring', 'out', 'the', 'balance', 'between', 'two', 'activities', 'exploration', 'and', 'exploitation'], ['both', 'are', 'necessary', 'but', 'it', 'can', 'be', 'too', 'much', 'of', 'a', 'good', 'thing'], ['consider', 'facit'], ['i', 'm', 'actually', 'old', 'enough', 'to', 'remember', 'them'], ['facit', 'was', 'a', 'fantastic', 'company'], ['they', 'were', 'born', 'deep', 'in', 'the', 'swedish', 'forest', 'and', 'they', 'made', 'the', 'best', 'mechanical', 'calculators', 'in', 'the', 'world'], ['everybody', 'used', 'them'], ['and', 'what', 'did', 'facit', 'do', 'when', 'the', 'electronic', 'calculator', 'came', 'along'], ['they', 'continued', 'doing', 'exactly', 'the', 'same']]


### Word2Vec - Continuous Bag-Of-Words (CBOW)

In [0]:
wv_cbow_model = Word2Vec(sentences=sentences, size=100, window=5, min_count=5, workers=4, sg=0)

In [0]:
similar_words=wv_cbow_model.wv.most_similar("man")
pprint.pprint(similar_words)

[('woman', 0.851108193397522),
 ('guy', 0.8076339364051819),
 ('lady', 0.7675459384918213),
 ('boy', 0.7619550228118896),
 ('girl', 0.7390323877334595),
 ('gentleman', 0.7282595634460449),
 ('soldier', 0.7208263874053955),
 ('poet', 0.7036651372909546),
 ('kid', 0.6826914548873901),
 ('rabbi', 0.6775092482566833)]


### Word2Vec - Skip Gram

In [0]:
wv_sg_model = Word2Vec(sentences=sentences, size=100, window=5, min_count=5, workers=4, sg=1)

In [0]:
similar_words=wv_sg_model.wv.most_similar("man")
pprint.pprint(similar_words)

[('guy', 0.7681315541267395),
 ('woman', 0.7646993398666382),
 ('soldier', 0.709970235824585),
 ('rabbi', 0.7043532133102417),
 ('boy', 0.7032051086425781),
 ('gentleman', 0.6925383806228638),
 ('policeman', 0.6811267137527466),
 ('handsome', 0.6787228584289551),
 ('comedian', 0.6779303550720215),
 ('cowboy', 0.677730917930603)]


## Word2Vec vs FastText

Word2Vec - Skipgram cannot find similar word "electrofishing" as "electrofishing" is not in the vocabulary.

In [0]:
similar_words=wv_sg_model.wv.most_similar("electrofishing")
pprint.pprint(similar_words)

KeyError: ignored

### FastText - Skip Gram

In [0]:
from gensim.models import FastText

In [0]:
ft_sg_model = FastText(sentences, size=100, window=5, min_count=5, workers=4, sg=1)

In [0]:
result=ft_sg_model.wv.most_similar("electrofishing")
pprint.pprint(result)

[('electrolux', 0.8047779202461243),
 ('electro', 0.7989339828491211),
 ('electric', 0.7973409295082092),
 ('electrolyte', 0.7957272529602051),
 ('electrochemical', 0.7689745426177979),
 ('electroshock', 0.7645504474639893),
 ('airbus', 0.7607172727584839),
 ('electrogram', 0.7588797807693481),
 ('gastric', 0.7578988671302795),
 ('electroencephalogram', 0.7539584636688232)]


### FastText - Continuous Bag-Of-Words (CBOW)

In [0]:
ft_cbow_model = FastText(sentences, size=100, window=5, min_count=5, workers=4, sg=0)

In [0]:
result=ft_cbow_model.wv.most_similar("electrofishing")
pprint.pprint(result)

[('electric', 0.9130792617797852),
 ('electro', 0.9014642238616943),
 ('electrolux', 0.8920943737030029),
 ('electron', 0.8828102350234985),
 ('electronic', 0.8791064023971558),
 ('electroshock', 0.8719875812530518),
 ('electrical', 0.8711882829666138),
 ('electrolyte', 0.8692057728767395),
 ('electrode', 0.8615459203720093),
 ('electromagnet', 0.8559844493865967)]


## King - Man + Woman = ?

Try both CBOW and Skip Gram model to calculate "King - Man + Woman = ?"

In [0]:
result = wv_cbow_model.wv.most_similar(positive=['woman', 'king'], negative=['man'], topn=1)
print(result)

[('president', 0.7746388912200928)]


In [0]:
result = wv_sg_model.wv.most_similar(positive=['woman', 'king'], negative=['man'], topn=1)
print(result)


[('luther', 0.6568899750709534)]


In [0]:
result = ft_cbow_model.wv.most_similar(positive=['woman', 'king'], negative=['man'], topn=1)
print(result)


[('kidding', 0.8817100524902344)]


In [0]:
result = ft_sg_model.wv.most_similar(positive=['woman', 'king'], negative=['man'], topn=1)
print(result)


[('jarring', 0.7067565321922302)]


This is not what we expected...Probably not enough data to answer as "Queen"

Let's  try with bigger sized data (Google has already trained Word2Vec with Google News data)


### Google's Pretrained Word2Vec (Google News)
[Link to Project](https://code.google.com/archive/p/word2vec/)


In [0]:
# Beware, this file is big (3.39GB) 
id2 = '1cOEYOQRd1VXi7ROShhqZbioCcePvgnR5'
downloaded = drive.CreateFile({'id':id2}) 
downloaded.GetContentFile('GoogleNews-vectors-negative300.bin')  

In [0]:
from gensim.models import KeyedVectors
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

filename = 'GoogleNews-vectors-negative300.bin'
gn_wv_model = KeyedVectors.load_word2vec_format(filename, binary=True)

In [0]:
result = gn_wv_model.most_similar(positive=['woman', 'king'], negative=['man'], topn=1)
print(result)

[('queen', 0.7118192911148071)]


# Exercise
**You need to implement a Lab2 chatbot that returns a duckduckgo search url with the similar words (retrieved by Google's Word2Vec)**
**(i.e. https://duckduckgo.com/?q=sydney+hotels+Hotel+motel+boutique_hotel)**


1.  Chatbot should start with saying "Welcome. Where would you like to search?"
2.  When search term (search term must be only one word) is entered by user, chatbot should return a search url with top 4 similar words (retrieved by Google's Word2Vec)
3.  After showing a search url, chatbot should ask whether user is satisfied with the result, "Are you satisfied with this result?"
4.  If user says "yes", then system says "Thank you! See you again" and system should be shut down.
5.  If user says "no", then system ask "Where would you like to search?" and goes back to process 2






## Example Communication (Between Lab2 Chatbot and you) 


```
Chatbot: Welcome. What would you like to search?
You: hotel
Chatbot: https://duckduckgo.com/?q=hotels+Hotel+motel+boutique_hotel
Chatbot: Are you satisfied with the result?
You: no
Chatbot: What would you like to search?
You: hospital
Chatbot: https://duckduckgo.com/?q=Hospital+hopsital+hosptial+hospitals
Chatbot: Are you satisfied with the result?
You: yes
Chatbot: Thank you! See you again.
```





Useful information: [API for Gensim Word2Vec](https://radimrehurek.com/gensim/models/word2vec.html)



In [7]:
# You should submit "ipynb" file (You can download it from "File" > "Download .ipynb") to Canvas
conda install -c anaconda gensim 

import nltk
nltk.download('punkt')

import pprint
import re
from lxml import etree
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize

from gensim.models import Word2Vec

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from gensim.models import KeyedVectors
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

filename = 'GoogleNews-vectors-negative300.bin'
gn_wv_model = KeyedVectors.load_word2vec_format(filename, binary=True)



print("Chatbot: Welcome. What would you like to search?")
word = input("You: ")
result = gn_wv_model.most_similar(positive = [word], topn=4)
print(result)

print("Chatbot: Are you satisfied with the result?")
satisfy = input("You:")

while satisfy != "yes":
  if satisfy == "no":
    print("Chatbot: What would you like to search?")
    word = input("You: ")
    result = gn_wv_model.most_similar(positive = [word], topn=4)
    
    link = "https://duckduckgo.com/?q="+"result[0][0]"+"+"+"result[1][0]"+"+"+"result[2][0]"+"+"+"result[0][0]"
    
    print("Chatbot: "+link)
    print("Chatbot: Are you satisfied with the result?")
    satisfy = input("You: ")
  else:
    print("Chatbot: Are you satisfied with the result?")
    satisfy = input("You: ")    

  
print("Chatbot: Thank you! See you again.")



SyntaxError: invalid syntax (<ipython-input-7-37e687f4afc6>, line 2)

# Extension

## Word2Vec with Tensorflow

If you want to implement Word2Vec with Tensorflow, here is a [sample code](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/examples/tutorials/word2vec/word2vec_basic.py) by [tensorflow](https://github.com/tensorflow).  

## Word Embedding Visual Inspector (WEVI)
If you would like to visualise how Word2Vec is learning, the following link is useful https://ronxin.github.io/wevi/